In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  #默认为'last'
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import time
import os
import sys
import random
import re
from numpy import linalg
np.set_printoptions(threshold=np.inf, precision=6, suppress=True)

In [2]:
%whos

Variable           Type                          Data/Info
----------------------------------------------------------
InteractiveShell   MetaHasTraits                 <class 'IPython.core.inte<...>eshell.InteractiveShell'>
NamespaceMagics    MetaHasTraits                 <class 'IPython.core.magi<...>mespace.NamespaceMagics'>
autopep8           module                        <module 'autopep8' from '<...>te-packages/autopep8.py'>
get_ipython        function                      <function get_ipython at 0x7f418b66ddc0>
getsizeof          builtin_function_or_method    <built-in function getsizeof>
json               module                        <module 'json' from '/hom<...>hon3.9/json/__init__.py'>
linalg             module                        <module 'numpy.linalg' fr<...>umpy/linalg/__init__.py'>
np                 module                        <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os                 module                        <module 'os' from '/home/<...>da3/

[Jupyter Notebook 有哪些奇技淫巧？](https://www.zhihu.com/question/266988943)

[Anaconda下的Jupyter Notebook中使用Cython](https://blog.csdn.net/dzhanghz/article/details/104403654)  

In [4]:
!which pip

/home/wenquanshan/miniconda3/bin/pip


In [5]:
!pip install Cython

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c3/8f/bb0a7182dc081fbc6608e98a8184970e7d903acfc1ec58680d46f5c915ce/Cython-0.29.32-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (2.0 MB)

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [7]:
%load_ext cython

In [8]:
%%cython -n add_cc
def add_c(a, b):
    for i in range(100000):
        c = a + b
    return c

In [9]:
%%cython -n add_cc2
cdef add_c2(a, b):
    for i in range(100000):
        c = a + b
    return c

/data1/home/wenquanshan/.cache/ipython/cython/add_cc2.c:1195:18: warning: ‘__pyx_f_7add_cc2_add_c2’ defined but not used [-Wunused-function]
 static PyObject *__pyx_f_7add_cc2_add_c2(PyObject *__pyx_v_a, PyObject *__pyx_v_b) {
                  ^~~~~~~~~~~~~~~~~~~~~~~


In [10]:
add_c(1,2)

3

## cython

## sys.modules

In [11]:
sys.modules # 所有模块

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module 'importlib._bootstrap' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module 'io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module 'importlib._bootstrap_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' from '/home/wenquanshan/miniconda3/lib/python3.9/codecs.py'>,
 'encodings.aliases': <module 'encodings.aliases' from '/home/wenquanshan/miniconda3/lib/python3.9/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/home/wenquanshan/miniconda3/lib/python3.9/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.u

In [12]:
sys.modules[add_c.__module__] # cython模块信息

<module 'add_cc' (/data1/home/wenquanshan/.cache/ipython/cython/add_cc.cpython-39-x86_64-linux-gnu.so)>

In [13]:
add_c.__module__ # cython模块名

'add_cc'

### 用-n参数指定编译之后的扩展模块名

In [35]:
%%cython -n add_d
cdef add_d(a, b):
    for i in range(100000):
        c = a + b
    return c

In [36]:
add_d(1,2)

3

### 用-a参数查看编译后的c语言程序

In [14]:
%%cython -a
a = 1.0
b = 2.0
c = a+b

### cdef

In [15]:
%%cython -a -n var1
cdef double a1 = 1.0
cdef double b1 = 2.0
cdef double c1 = a1+b1

In [16]:
a1
type(a1)

NameError: name 'a1' is not defined

In [17]:
%%cython -a
cdef double s = 0.0
a = 3.0
s = s + a
s = s + <double>a


In [30]:
%%cython -a
cdef list clist = [1,2,3]
cdef int cindex=0
clist[cindex]

plist = [1,2,3]

pindex=0
plist[pindex]

NameError: name 'clist' is not defined

## cython2

In [23]:
import numpy as np

In [24]:
np.random.seed(42)
X = np.random.rand(200, 3)

In [25]:
def pairwise_dist_python(X):
    m, n = X.shape
    D = np.empty((m, m), dtype=np.float32)
    for i in range(m):
        for j in range(i, m):
            d = 0.0
            for k in range(n):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = D[j, i] = d ** 0.5
    return D

In [26]:
from scipy.spatial.distance import pdist, squareform

In [27]:
%%timeit
squareform(pdist(X))

126 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
%%timeit
pairwise_dist_python(X)

38.1 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
np.allclose(squareform(pdist(X)), pairwise_dist_python(X))

True

In [30]:
%%cython
import numpy as np
def pairwise_dist_python_cython(X):
    m, n = X.shape
    D = np.empty((m, m), dtype=np.float32)
    for i in range(m):
        for j in range(i, m):
            d = 0.0
            for k in range(n):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = D[j, i] = d ** 0.5
    return D

In [31]:
%%timeit
pairwise_dist_python_cython(X)

33 ms ± 553 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
np.allclose(squareform(pdist(X)), pairwise_dist_python_cython(X))

True

In [35]:
%%cython
import numpy as np
import cython
@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_dist_python_cython2(double[:,::1] X):
    cdef int m, n, i, j, k
    cdef double tmp, d
    m, n = X.shape[0], X.shape[1]
    
    cdef double[:,::1] D = np.empty((m, m), dtype=np.float64)
    for i in range(m):
        for j in range(i, m):
            d = 0.0
            for k in range(n):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = D[j, i] = d ** 0.5
    return D

In [36]:
%%timeit
pairwise_dist_python_cython2(X)

1.27 ms ± 28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [37]:
np.allclose(squareform(pdist(X)), pairwise_dist_python_cython2(X))

True

## loop

In [18]:
def add_python(a,b):
    for i in range(1000000):
        c = a + b
    return a + b

In [20]:
a = 1
b = 1

In [21]:
%%timeit
for i in range(1):
    add_c(a,b)

717 µs ± 4.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit
for i in range(1):
    add_python(a,b)

33.6 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## fib

[Jupyter Notebook 有哪些奇技淫巧？](https://www.zhihu.com/question/266988943)  
可以使用单次运行的 ％time 或 ％timeit 来计算大量时间，并获得平均值和标准偏差。所以这对于这些简单的函数很有用，那么调用其他函数的函数呢？  
这里有 ％prun，我们创建了一个虚函数，能够调用 fib1（）很多时间。可以看到循环确实花了一些时间，但大部分时间花在 fib1（）上。


In [6]:
def fib1(n):
    a,b=0,1
    while b<n:
        a,b = b,a+b

In [7]:
%%cython
def fib2(n):
    a,b=0,1
    while b<n:
        a,b = b,a+b

In [10]:
%%cython
def fib3(int n): # 指定变量类型对加快速度非常有帮助
    cdef int a = 0
    cdef int b = 1
    cdef int t = 0
    while b<n:
        t = a
        a = b
        b = t + b

In [11]:
%timeit fib1(500)
%timeit fib2(500)
%timeit fib3(500) 

777 ns ± 69.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
393 ns ± 20.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
38.3 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [81]:
def dummy1(n):
    for i in range(10000000):
        fib1(n)

In [85]:
def dummy2(n):
    for i in range(10000000):
        fib2(n)

In [87]:
def dummy3(n):
    for i in range(10000000):
        fib3(n)

In [83]:
%prun dummy1(50)

In [88]:
%prun dummy2(50)

In [89]:
%prun dummy3(50)

$$ P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)} $$
$a|b$  
$a\mid b$

In [90]:
#Note that http urls will not be displayed. Only https are allowed inside the Iframe
from IPython.display import IFrame
IFrame('https://en.wikipedia.org/wiki/HTTPS', width=800, height=450)


In [91]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1406.2661.pdf', width=800, height=450)

## other

In [12]:
%%cython -n func3
def app(int n):
    cdef int i=1
    cdef long sum=0
    while i<n:
        sum += i
        i+=1
    return sum


In [14]:
a = app(50000)
print(a)

1249975000


In [15]:
def app_python(n):
    sum = 0
    for i in range(n):
        sum += i
    return sum

In [16]:
app_python(50000)

1249975000

In [19]:
%timeit app(100)
%timeit app_python(100)

88.8 ns ± 4.54 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
4.73 µs ± 442 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
